In [12]:
import yfinance as yf
import requests
import os
#yfinance is a widely used Python library that provides a convenient and efficient way to access and download financial data from Yahoo Finance.
import pandas as pd
import pandas_ta as ta
#Pandas Technical Analysis (pandas_ta) is a Python library designed for technical analysis of financial data. It extends the popular Pandas library, providing a wide array of technical indicators and utility functions directly accessible from Pandas DataFrames.
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
import warnings
import random
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv('./vars.env')
api_key=os.getenv("API_KEY")

In [3]:
# Define parameters
SYMBOL = 'QQQ'
RSI_PERIOD = 14
OVERBOUGHT = 70
OVERSOLD = 30
REVERSAL_THRESHOLD = 0.005  # 0.5% price move to confirm reversal
REVERSAL_WINDOW = 5  # Number of 5-minute bars to check for reversal
LOOKAHEAD_MINUTES = [10, 20, 30]  # Minutes to measure price change
START_DATE = (datetime.today() - timedelta(days=60)).strftime('%Y-%m-%d')  # Reduced to 60 days
END_DATE = datetime.today().strftime('%Y-%m-%d')
CACHE_FILE = 'qqq_intraday_data.csv'

In [14]:
# finnhub free account does not offer access to ticker QQQ
# def fetch_intraday_data(symbol, start_date, end_date, interval='5m', token=api_key):
#     """
#     Download historical stock price data using Finnhub API.
#     interval: '1m', '5m', '15m', '30m', '60m', 'D', etc.
#     """
#     # Finnhub intervals are '1', '5', '15', '30', '60' for intraday (in minutes)
#     interval_map = {'1m': '1', '5m': '5', '15m': '15', '30m': '30', '60m': '60', 'D': 'D'}
#     finnhub_interval = interval_map.get(interval, '5')
#
#     start_dt = datetime.strptime(start_date, '%Y-%m-%d')
#     end_dt = datetime.strptime(end_date, '%Y-%m-%d')
#     all_data = []
#     cur_start = start_dt
#
#     while cur_start < end_dt:
#         cur_end = min(cur_start + timedelta(days=30), end_dt)
#         from_ts = int(cur_start.timestamp())
#         to_ts = int(cur_end.timestamp())
#
#         url = 'https://finnhub.io/api/v1/stock/candle'
#         params = {
#             'symbol': symbol,
#             'resolution': finnhub_interval,
#             'from': from_ts,
#             'to': to_ts,
#             'token': token
#         }
#         for attempt in range(3):
#             try:
#                 r = requests.get(url, params=params)
#                 r.raise_for_status()
#                 result = r.json()
#                 if result.get('s') != 'ok':
#                     print(f"No intraday data for {symbol} from {cur_start} to {cur_end}")
#                     break
#                 df = pd.DataFrame({
#                     'time': pd.to_datetime(result['t'], unit='s'),
#                     'Open': result['o'],
#                     'High': result['h'],
#                     'Low': result['l'],
#                     'Close': result['c'],
#                     'Volume': result['v'],
#                 }).set_index('time')
#                 all_data.append(df)
#                 break
#             except Exception as e:
#                 print(f"Error fetching {symbol}: {e}")
#                 if attempt < 2:
#                     wait = 2 ** attempt + random.uniform(0, 2)
#                     print(f"Retrying after {wait:.2f} seconds...")
#                     time.sleep(wait)
#                 else:
#                     print(f"Max retries exceeded for {symbol} chunk {cur_start} to {cur_end}. Skipping.")
#         cur_start = cur_end + timedelta(days=1)
#         # avoid hammering the server
#         time.sleep(random.uniform(1, 2))
#
#     if all_data:
#         data = pd.concat(all_data)
#         data = data[~data.index.duplicated(keep='first')]
#         return data
#     else:
#         print("No intraday data available. Falling back to daily data.")
#         # Try daily candles
#         try:
#             from_ts = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
#             to_ts = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp())
#             params_daily = {
#                 'symbol': symbol,
#                 'resolution': 'D',
#                 'from': from_ts,
#                 'to': to_ts,
#                 'token': token
#             }
#             r = requests.get(url, params=params_daily)
#             r.raise_for_status()
#             result = r.json()
#             if result.get('s') == 'ok':
#                 df = pd.DataFrame({
#                     'time': pd.to_datetime(result['t'], unit='s'),
#                     'Open': result['o'],
#                     'High': result['h'],
#                     'Low': result['l'],
#                     'Close': result['c'],
#                     'Volume': result['v'],
#                 }).set_index('time')
#                 return df
#             else:
#                 print("No daily data found either.")
#                 return pd.DataFrame()
#         except Exception as e:
#             print(f"Error fetching daily data: {e}")
#             return pd.DataFrame()

In [18]:
df = fetch_intraday_data(SYMBOL, START_DATE, END_DATE, interval='5m', token=api_key)

Error fetching QQQ: 403 Client Error: Forbidden for url: https://finnhub.io/api/v1/stock/candle?symbol=QQQ&resolution=5&from=1745384400&to=1747976400&token=d1c7ja1r01qre5ak6s9gd1c7ja1r01qre5ak6sa0
Retrying after 2.49 seconds...
Error fetching QQQ: 403 Client Error: Forbidden for url: https://finnhub.io/api/v1/stock/candle?symbol=QQQ&resolution=5&from=1745384400&to=1747976400&token=d1c7ja1r01qre5ak6s9gd1c7ja1r01qre5ak6sa0
Retrying after 3.74 seconds...


KeyboardInterrupt: 

In [28]:
def safe_download(symbol, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            data = yf.download(symbol, period="60d", interval="5m", progress=False)
            if data.empty:
                print(f"No data for {symbol}")
            return data
        except Exception as e:
            if "rate limit" in str(e).lower() or "too many requests" in str(e).lower():
                wait = 2 ** retries * 10 + random.uniform(5, 20)
                print(f"Rate limited for {symbol}. Waiting {wait:.1f} seconds.")
                time.sleep(wait)
                retries += 1
            else:
                print(f"Other error for {symbol}: {e}")
                break
    print(f"Max retries exceeded for {symbol}")
    return None

# Example usage:
symbols = ['QQQ']
for symbol in symbols:
    df = safe_download(symbol)
    time.sleep(random.uniform(10, 30))


1 Failed download:
['QQQ']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


No data for QQQ


In [19]:
df.head()

""


In [ ]:
# Function to calculate RSI and identify signals
def calculate_rsi_signals(data):
    # Calculate RSI using pandas_ta
    data['RSI'] = ta.rsi(data['Close'], length=RSI_PERIOD)
    # Identify overbought and oversold signals
    data['Overbought'] = data['RSI'] > OVERBOUGHT
    data['Oversold'] = data['RSI'] < OVERSOLD
    # Mark signal points (first occurrence after crossing threshold)
    data['Signal'] = None

    #detect the momentum when the signal occurs (this does not consider if the RSI remains 'Overbought'/'Oversold' for several consecutive time windows
    for i in range(1, len(data)):
        if data['Overbought'].iloc[i] and not data['Overbought'].iloc[i-1]:
            data.loc[data.index[i], 'Signal'] = 'Sell'
        elif data['Oversold'].iloc[i] and not data['Oversold'].iloc[i-1]:
            data.loc[data.index[i], 'Signal'] = 'Buy'
    return data

# Function to detect reversals
def detect_reversals(data):
    data['Reversal'] = False
    for idx, row in data[data['Signal'].notnull()].iterrows(): #loop through all rows there is a signal
        signal_type = row['Signal']
        signal_price = row['Close'] #the close price when signal event occurs, used as benchmark
        # Get next REVERSAL_WINDOW bars
        end_idx = data.index.get_loc(idx) + REVERSAL_WINDOW
        if end_idx >= len(data):
            continue
        window_data = data.iloc[data.index.get_loc(idx)+1:end_idx+1]
        if signal_type == 'Sell':
            # Bearish reversal: price drops by threshold
            if window_data['Close'].min() <= signal_price * (1 - REVERSAL_THRESHOLD):
                data.loc[idx, 'Reversal'] = True
        elif signal_type == 'Buy':
            # Bullish reversal: price rises by threshold
            if window_data['Close'].max() >= signal_price * (1 + REVERSAL_THRESHOLD):
                data.loc[idx, 'Reversal'] = True
    return data

# Function to calculate price changes after signals
def calculate_price_changes(data, lookahead_days):
    results = []
    for idx, row in data[data['Signal'].notnull()].iterrows():
        signal_date = idx
        signal_price = row['Close']
        signal_type = row['Signal']
        reversal = row['Reversal']
        for days in lookahead_days:
            # Find price after N days
            target_date = signal_date + timedelta(days=days)
            # Get closest trading day after target_date
            future_data = data[data.index >= target_date].head(1)
            if not future_data.empty:
                future_price = future_data['Close'].iloc[0]
                price_change = (future_price - signal_price) / signal_price * 100
                results.append({
                    'Signal_Date': signal_date,
                    'Signal_Type': signal_type,
                    'Reversal': reversal,
                    'Days_Ahead': days,
                    'Price_Change (%)': price_change
                })
    return pd.DataFrame(results)

In [ ]:
# Main analysis function
def analyze_rsi_reversals():
    # Fetch data
    print("Fetching data...")
    data = fetch_intraday_data(SYMBOL, START_DATE, END_DATE)
    if data.empty:
        print("No data available. Exiting.")
        return

    # Calculate RSI and signals
    print("Calculating RSI and signals...")
    data = calculate_rsi_signals(data)

    # Detect reversals
    print("Detecting reversals...")
    data = detect_reversals(data)

    # Calculate price changes
    print("Calculating price changes...")
    price_changes = calculate_price_changes(data, LOOKAHEAD_DAYS)

    # Analyze results
    print("\nAnalysis Results:")
    # Reversal probability
    signals = data[data['Signal'].notnull()]
    if not signals.empty:
        reversal_prob = signals.groupby('Signal')['Reversal'].mean() * 100
        print("\nReversal Probability (%):")
        print(reversal_prob)

    # Price change statistics
    if not price_changes.empty:
        print("\nAverage Price Change (%) After Signals:")
        price_stats = price_changes.groupby(['Signal_Type', 'Days_Ahead'])['Price_Change (%)'].agg(['mean', 'std', 'count'])
        print(price_stats)

        # Plot price change distributions
        plt.figure(figsize=(12, 6))
        for signal in ['Buy', 'Sell']:
            for days in LOOKAHEAD_DAYS:
                subset = price_changes[(price_changes['Signal_Type'] == signal) & (price_changes['Days_Ahead'] == days)]
                plt.hist(subset['Price_Change (%)'], bins=20, alpha=0.5, label=f'{signal} {days}d')
        plt.title('Distribution of Price Changes After RSI Signals')
        plt.xlabel('Price Change (%)')
        plt.ylabel('Frequency')
        plt.legend()
        plt.grid(True)
        plt.show()

    # Plot RSI and signals
    plt.figure(figsize=(14, 8))
    plt.subplot(2, 1, 1)
    plt.plot(data.index, data['Close'], label='Close Price')
    buy_signals = data[data['Signal'] == 'Buy']
    sell_signals = data[data['Signal'] == 'Sell']
    plt.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='g', label='Buy Signal', s=100)
    plt.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='r', label='Sell Signal', s=100)
    plt.title(f'{SYMBOL} Price and RSI Signals')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)

    plt.subplot(2, 1, 2)
    plt.plot(data.index, data['RSI'], label='RSI')
    plt.axhline(OVERBOUGHT, color='r', linestyle='--', alpha=0.5)
    plt.axhline(OVERSOLD, color='g', linestyle='--', alpha=0.5)
    plt.scatter(buy_signals.index, buy_signals['RSI'], marker='^', color='g', label='Buy Signal', s=100)
    plt.scatter(sell_signals.index, sell_signals['RSI'], marker='v', color='r', label='Sell Signal', s=100)
    plt.title('RSI')
    plt.ylabel('RSI')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    analyze_rsi_reversals()